In [7]:
import json
import pandas as pd

# File paths for the two input JSON files
file_path_rf = '/u/44/yangz2/unix/Documents/vs_code_programs/joint_angles/data/20240920T201312Z_175030000302_imu_stream.json'
file_path_ru = '/u/44/yangz2/unix/Documents/vs_code_programs/joint_angles/data/20240920T201312Z_240930000239_imu_stream.json'

def load_imu_data(file_path):
    with open(file_path, 'r') as f:
        data = json.load(f)['data']
    return data

# Extract the IMU data (timestamps, accelerometer, and gyroscope readings)
def extract_imu_data(data):
    timestamps = []
    acc_x, acc_y, acc_z = [], [], []
    gyr_x, gyr_y, gyr_z = [], [], []
    
    for entry in data:
        imu = entry['imu']  # Access the 'imu' key
        timestamps.append(imu['Timestamp'])
        
        # Extract the first accelerometer and gyroscope readings
        acc_array = imu['ArrayAcc'][0]
        gyro_array = imu['ArrayGyro'][0]
        
        acc_x.append(acc_array['x'])
        acc_y.append(acc_array['y'])
        acc_z.append(acc_array['z'])
        
        gyr_x.append(gyro_array['x'])
        gyr_y.append(gyro_array['y'])
        gyr_z.append(gyro_array['z'])
    
    return pd.DataFrame({
        'timestamp': timestamps,
        'Acc_X': acc_x,
        'Acc_Y': acc_y,
        'Acc_Z': acc_z,
        'Gyro_X': gyr_x,
        'Gyro_Y': gyr_y,
        'Gyro_Z': gyr_z
    })

# Load and extract data for RF (Right Forearm)
rf_data = load_imu_data(file_path_rf)
rf_df = extract_imu_data(rf_data)
rf_df.rename(columns={
    'Acc_X': 'RF_IMU9_Acc_X', 'Acc_Y': 'RF_IMU9_Acc_Y', 'Acc_Z': 'RF_IMU9_Acc_Z',
    'Gyro_X': 'RF_IMU9_Gyro_X', 'Gyro_Y': 'RF_IMU9_Gyro_Y', 'Gyro_Z': 'RF_IMU9_Gyro_Z'
}, inplace=True)

# Load and extract data for RU (Right Upper Arm)
ru_data = load_imu_data(file_path_ru)
ru_df = extract_imu_data(ru_data)
ru_df.rename(columns={
    'Acc_X': 'RU_IMU9_Acc_X', 'Acc_Y': 'RU_IMU9_Acc_Y', 'Acc_Z': 'RU_IMU9_Acc_Z',
    'Gyro_X': 'RU_IMU9_Gyro_X', 'Gyro_Y': 'RU_IMU9_Gyro_Y', 'Gyro_Z': 'RU_IMU9_Gyro_Z'
}, inplace=True)

# Merge the two datasets on the 'timestamp' column
merged_df = pd.merge(ru_df, rf_df, on='timestamp', suffixes=('_RU', '_RF'))

# Save the merged data to a CSV file
output_csv_path = 'output_ru_rf.csv'
merged_df.to_csv(output_csv_path, index=False)

print(f"CSV file saved to {output_csv_path}")


CSV file saved to output_ru_rf.csv
